In [151]:
#General libraries
import pandas as pd
import numpy as np
import glob as glob
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

#Libraries for feature engineering
from astral.sun import sun
from astral import LocationInfo
from workalendar.europe import Germany
from scipy.stats import entropy, zscore


In [152]:
def is_daytime(timestamp):
    s = sun(location.observer, date=timestamp.date())
    return s['sunrise'].time() < timestamp.time() < s['sunset'].time()

def is_holiday(timestamp):
    return cal.is_working_day(timestamp)

def get_season(timestamp):
    if (timestamp.month > 11 or timestamp.month < 3):
        return 'WINTER'
    elif (timestamp.month == 3 or timestamp.month <=5):
        return 'SPRING'
    elif (timestamp.month >=6 and timestamp.month <=9):
        return 'SUMMER'
    else:
        return 'FALL'

def calendar_features(df, quarter_hour=True, hour=True, weekday=True, month=True, quarter=True):
    features = df.copy()
    columns = []
    if quarter_hour:
        features['QUARTERHOUR'] = features.timestamp.dt.minute
        columns.append('QUARTERHOUR')
    if hour:
        features['HOUR'] = features.timestamp.dt.hour
        columns.append('HOUR')
    if weekday:
        features['WEEKDAY'] = features.timestamp.dt.dayofweek
        columns.append('WEEKDAY')
    if month:
        features['MONTH'] = features.timestamp.dt.month
        columns.append('MONTH')
    if quarter:
        features['QUARTER'] = features.timestamp.dt.quarter
        columns.append('QUARTER')

    dummies = pd.get_dummies(features[columns], columns=columns)

    return dummies

def window_metrics_15m(df, target, lag=True, ma=True, maxi=True, mini=True, suma=True, diff=True, entropy=False, zscore=False,
                       cos=False, freq=[32, 96, 672]):

    features = df.copy()


    for i in range(1,25):

        if i < 4:

            if lag:
                features['LAG_'+str((i)*15)+'MIN'] = features[target].shift((i))
                features['LAG_'+str(i)+'H'] = features[target].shift((i)*4)
                features['LAG_'+str(i)+'D'] = features[target].shift((i)*96)
            if ma:
                features['MA_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').mean()
                features['MA_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').mean()
                features['MA_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').mean()
            if maxi:
                features['MAX_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').max()
                features['MAX_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').max()
                features['MAX_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').max()
            if mini:
                features['MIN_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').min()
                features['MIN_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').min()
                features['MIN_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').min()
            if suma:
                features['SUM_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').sum()
                features['SUM_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').sum()
                features['SUM_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').sum()
            if diff:
                features['DIFF_96_'+str((i)*15)+'MIN'] = features[target].shift((i)).diff(96)
                features['DIFF_96_'+str(i)+'H'] = features[target].shift((i)*4).diff(96)
                features['DIFF_96_'+str(i)+'D'] = features[target].shift((i)*96).diff(96)
                features['DIFF_672_'+str((i)*15)+'MIN'] = features[target].shift((i)).diff(672)
                features['DIFF_672_'+str(i)+'H'] = features[target].shift((i)*4).diff(672)
                features['DIFF_672_'+str(i)+'D'] = features[target].shift((i)*96).diff(672)
                features['DIFF_96_672_'+str((i)*15)+'MIN'] = features[target].shift((i)).diff(96).diff(672)
                features['DIFF_96_672_'+str(i)+'H'] = features[target].shift((i)*4).diff(96).diff(672)
                features['DIFF_96_672_'+str(i)+'D'] = features[target].shift((i)*96).diff(96).diff(672)
            if entropy:
                features['ENTROPY_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
                features['ENTROPY_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
                features['ENTROPY_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
            if zscore:
                features['ZSCORE_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').apply(lambda x: zscore(x)[-1], raw=True)
                features['ZSCORE_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').apply(lambda x: zscore(x)[-1], raw=True)
                features['ZSCORE_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').apply(lambda x: zscore(x)[-1], raw=True)
            if cos:
                for k in freq:
                    features['COS_'+str(k)+'_LAG_'+str((i)*15)+'MIN'] = np.cos(features[target].shift((i))/k)
                    features['COS_'+str(k)+'_LAG_'+str(i)+'H'] = np.cos(features[target].shift((i)*4)/k)
                    features['COS_'+str(k)+'_LAG_'+str(i)+'D'] = np.cos(features[target].shift((i)*96)/k)

        elif i > 7:

            if lag:
                features['LAG_'+str(i)+'H'] = features[target].shift((i)*4)
            if ma:
                features['MA_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').mean()
            if maxi:
                features['MAX_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').max()
            if mini:
                features['MIN_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').min()
            if suma:
                features['SUM_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').sum()
            if diff:
                features['DIFF_96_'+str(i)+'H'] = features[target].shift((i)*4).diff(96)
                features['DIFF_672_'+str(i)+'H'] = features[target].shift((i)*4).diff(672)
                features['DIFF_96_672_'+str(i)+'H'] = features[target].shift((i)*4).diff(96).diff(672)
            if entropy:
                features['ENTROPY_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
            if zscore:
                features['ZSCORE_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').apply(lambda x: zscore(x)[-1], raw=True)
            if cos:
                for k in freq:
                    features['COS_'+str(k)+'_LAG_'+str(i)+'H'] = np.cos(features[target].shift((i)*4)/k)

        else:
            if lag:
                features['LAG_'+str(i)+'H'] = features[target].shift((i)*4)
                features['LAG_'+str(i)+'D'] = features[target].shift((i)*96)
            if ma:
                features['MA_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').mean()
                features['MA_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').mean()
            if maxi:
                features['MAX_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').max()
                features['MAX_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').max()
            if mini:
                features['MIN_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').min()
                features['MIN_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').min()
            if suma:
                features['SUM_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').sum()
                features['SUM_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').sum()
            if diff:
                features['DIFF_96_'+str(i)+'H'] = features[target].shift((i)*4).diff(96)
                features['DIFF_96_'+str(i)+'D'] = features[target].shift((i)*96).diff(96)
                features['DIFF_672_'+str(i)+'H'] = features[target].shift((i)*4).diff(672)
                features['DIFF_672_'+str(i)+'D'] = features[target].shift((i)*96).diff(672)
                features['DIFF_96_672_'+str(i)+'H'] = features[target].shift((i)*4).diff(96).diff(672)
                features['DIFF_96_672_'+str(i)+'D'] = features[target].shift((i)*96).diff(96).diff(672)
            if entropy:
                features['ENTROPY_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
                features['ENTROPY_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
            if zscore:
                features['ZSCORE_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').apply(lambda x: zscore(x)[-1], raw=True)
                features['ZSCORE_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').apply(lambda x: zscore(x)[-1], raw=True)
            if cos:
                for k in freq:
                    features['COS_'+str(k)+'_LAG_'+str(i)+'H'] = np.cos(features[target].shift((i)*4)/k)
                    features['COS_'+str(k)+'_LAG_'+str(i)+'D'] = np.cos(features[target].shift((i)*96)/k)
    return features.drop(columns=[target])


def window_metrics_15m_SM(df, target, lag=True, ma=True, maxi=True, mini=True, suma=True, diff=True, entropy=False, zscore=False, end=25):

    features = df.copy()


    for i in range(1, end):

        if lag:
            features['LAG_'+str((i)*15)+'MIN'] = features[target].shift((i))
            features['LAG_'+str(i)+'H'] = features[target].shift((i)*4)
            if i == 7:
                features['LAG_'+str(i)+'D'] = features[target].shift((i)*96)
        if ma:
            features['MA_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').mean()
            features['MA_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').mean()
            if i == 7:
                features['MA_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').mean()
        if maxi:
            features['MAX_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').max()
            features['MAX_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').max()
            if i == 7:
                features['MAX_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').max()
        if mini:
            features['MIN_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').min()
            features['MIN_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').min()
            if i == 7:
                features['MIN_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').min()
        if suma:
            features['SUM_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').sum()
            features['SUM_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').sum()
            if i == 7:
                features['SUM_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').sum()
        if diff:
            features['DIFF_96_'+str((i)*15)+'MIN'] = features[target].shift((i)).diff(96)
            features['DIFF_96_'+str(i)+'H'] = features[target].shift((i)*4).diff(96)
            if i == 7:
                features['DIFF_96_'+str(i)+'D'] = features[target].shift((i)*96).diff(96)
            features['DIFF_672_'+str((i)*15)+'MIN'] = features[target].shift((i)).diff(672)
            features['DIFF_672_'+str(i)+'H'] = features[target].shift((i)*4).diff(672)
            if i == 7:
                features['DIFF_672_'+str(i)+'D'] = features[target].shift((i)*96).diff(672)
            features['DIFF_96_672_'+str((i)*15)+'MIN'] = features[target].shift((i)).diff(96).diff(672)
            features['DIFF_96_672_'+str(i)+'H'] = features[target].shift((i)*4).diff(96).diff(672)
            if i == 7:
                features['DIFF_96_672_'+str(i)+'D'] = features[target].shift((i)*96).diff(96).diff(672)
        if entropy:
            features['ENTROPY_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
            features['ENTROPY_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
            if i == 7:
                features['ENTROPY_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').apply(lambda x: entropy(np.histogram(x, bins='fd')[0]), raw=True)
        if zscore:
            features['ZSCORE_'+str((i+1)*15)+'MIN'] = features[target].rolling((i+1), closed='left').apply(lambda x: zscore(x)[-1], raw=True)
            features['ZSCORE_'+str(i)+'H'] = features[target].rolling((i)*4, closed='left').apply(lambda x: zscore(x)[-1], raw=True)
            if i == 7:
                features['ZSCORE_'+str(i)+'D'] = features[target].rolling((i)*96, closed='left').apply(lambda x: zscore(x)[-1], raw=True)
        # if cos:
        #     for k in freq:
        #         features['COS_'+str(k)+'_LAG_'+str((i)*15)+'MIN'] = np.cos(features[target].shift((i))/k)
        #         features['COS_'+str(k)+'_LAG_'+str(i)+'H'] = np.cos(features[target].shift((i)*4)/k)
        #         if i == 7:
        #             features['COS_'+str(k)+'_LAG_'+str(i)+'D'] = np.cos(features[target].shift((i)*96)/k)

    return features.drop(columns=[target])

In [153]:
df = pd.read_excel('dataset_real_003.xlsx', sheet_name=0)
df2 = pd.read_excel('dataset_real_003.xlsx', sheet_name=1)
data = pd.merge(df, df2, on='Date', how='outer')
data['consumption'] = data['Consumption (kWh)_x'] + data['Consumption (kWh)_y']
data = data[['Date', 'consumption']]
data.columns = ['timestamp', 'consumption']
df = data.copy()

In [154]:
# Define the location of interest (e.g. New York)
location = LocationInfo("Werther", "Germany", "Europe/Berlin", 52.0794, 8.4399)
df['is_daytime'] = df['timestamp'].apply(is_daytime).astype(int)

cal = Germany()
df['holiday'] = df['timestamp'].apply(is_holiday)
df['holiday'] = df['holiday'].astype(int)
df['holiday'] = abs(df['holiday'] - 1)

df['SEASON'] = df['timestamp'].apply(get_season)
df = pd.get_dummies(df, columns=['SEASON'])

calendar_dummies = calendar_features(df)
df = pd.concat([df, calendar_dummies], axis=1)

df.index = df['timestamp']
df.drop(columns='timestamp', inplace=True)

# window_features = window_metrics_15m_SM(df[['consumption']], 'consumption', lag=True, ma=False, maxi=False, mini=False, suma=False, diff=False, end=4+1)
# df = pd.concat([df, window_features], axis=1)

# df["MA_1H"] = df["consumption"].rolling(4, closed="left").mean()
df["MA_3H"] = df["consumption"].rolling(12, closed="left").mean()

# df["LAG_15MIN"] = df["consumption"].shift(1)

# # df["DIFF_15-30MIN"] = df["consumption"].shift(1).diff(1)
# df["DIFFSIGN_15-30MIN"] = np.sign(df["consumption"].shift(1).diff(1))
# # df["DIFF_30-45MIN"] = df["consumption"].shift(2).diff(1)
# df["DIFFSIGN_30-45MIN"] = np.sign(df["consumption"].shift(2).diff(1))
# # df["DIFF_45-60MIN"] = df["consumption"].shift(3).diff(1)
# df["DIFFSIGN_45-60MIN"] = np.sign(df["consumption"].shift(3).diff(1))

# df["MAX_1H"] = df["consumption"].rolling(4, closed="left").max()
df["MAX_5H"] = df["consumption"].rolling(20, closed="left").max()
df["MAX_3H"] = df["consumption"].rolling(12, closed="left").max()
# df["MIN_1H"] = df["consumption"].rolling(4, closed="left").min()
df["MIN_5H"] = df["consumption"].rolling(20, closed="left").min()
df["MIN_3H"] = df["consumption"].rolling(12, closed="left").min()

# df["SUM_1H"] = df["consumption"].rolling(4, closed="left").sum()
df["SUM_5H"] = df["consumption"].rolling(20, closed="left").sum()
df["SUM_3H"] = df["consumption"].rolling(12, closed="left").sum()


df['WEEKEND'] = (df.index.dayofweek >= 5)

df['WORKDAY'] = 1
df.loc[(df.WEEKEND == 1) | (df.holiday == 1), 'WORKDAY'] = 0

df = df.dropna()

df

,consumption,is_daytime,holiday,SEASON_FALL,SEASON_SPRING,SEASON_SUMMER,SEASON_WINTER,QUARTERHOUR_0,QUARTERHOUR_15,QUARTERHOUR_30,...,QUARTER_4,MA_3H,MAX_5H,MAX_3H,MIN_5H,MIN_3H,SUM_5H,SUM_3H,WEEKEND,WORKDAY
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 05:00:00,84.725,0,1,0,0,0,1,1,0,0,...,0,81.210833,89.670,89.670,77.280,77.280,1629.995,974.530,False,0
2021-01-01 05:15:00,84.725,0,1,0,0,0,1,0,1,0,...,0,81.210833,89.670,89.670,77.280,77.280,1634.940,974.530,False,0
2021-01-01 05:30:00,82.225,0,1,0,0,0,1,0,0,1,...,0,81.622917,89.670,89.670,77.280,77.280,1634.940,979.475,False,0
2021-01-01 05:45:00,84.725,0,1,0,0,0,1,0,0,0,...,0,82.035000,89.670,89.670,77.280,77.280,1639.885,984.420,False,0
2021-01-01 06:00:00,84.725,0,1,0,0,0,1,1,0,0,...,0,82.447083,89.670,89.670,77.280,77.280,1639.885,989.365,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-19 22:45:00,416.823,0,0,0,0,1,0,0,0,0,...,0,381.675083,411.477,411.477,353.724,353.724,7642.745,4580.101,False,1
2022-09-19 23:00:00,409.908,0,0,0,0,1,0,1,0,0,...,0,386.933333,416.823,416.823,353.724,358.903,7656.893,4643.200,False,1
2022-09-19 23:15:00,417.320,0,0,0,0,1,0,0,1,0,...,0,390.328000,416.823,416.823,353.724,358.903,7696.077,4683.936,False,1


In [155]:
df.to_csv("data_prep_MA3H-MINMAX5n1-SUM35.csv", index=True)